In [ ]:

dataset_path = 'datasets'
model_path = 'models'
train_dir = 'train'
test_dir = 'test'

In [57]:
import os
import shutil
import random

def split_dataset(class_names, split_ratio=0.8):
    for class_name in class_names:
        os.makedirs(os.path.join(dataset_path, train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(dataset_path, test_dir, class_name), exist_ok=True)

        dataset_files = os.listdir(os.path.join(dataset_path, class_name))

        random.shuffle(dataset_files)

        train_size = int(len(dataset_files) * split_ratio)

        train_files = dataset_files[:train_size]
        test_files = dataset_files[train_size:]

        for file in train_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(dataset_path, train_dir, class_name, file))

        for file in test_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(dataset_path, test_dir, class_name, file))

        print(f"Dataset {class_name} split into {len(train_files)} training files and {len(test_files)} test files.")

split_dataset(["cothe", "khong", "nhung"], 0.7)

Dataset cothe split into 65 training files and 28 test files.
Dataset khong split into 41 training files and 18 test files.
Dataset nhung split into 69 training files and 30 test files.


In [108]:
import preprocessing


def prepare_train_data(idx, classname, n_mfcc):
    file_paths = [os.path.join(dataset_path, train_dir, classname, i) for i in os.listdir(
        os.path.join(dataset_path, train_dir, classname)) if i.endswith('.wav')]
    data = [preprocessing.get_mfcc(file_path, n_mfcc) for file_path in file_paths]
    labels = [idx for _ in range(len(file_paths))]
    return data, labels

In [ ]:
from hmmlearn.hmm import GMMHMM
import numpy as np
import matplotlib.pyplot as plt

def train(class_name, data, number_of_states= 4, n_mix = 1, n_iter = 300): 
    start_prob = np.full(number_of_states, 0.0)
    start_prob[0] = 1.0
    trans_matrix = np.full((number_of_states, number_of_states), 0.0)
    p = 0.5
    np.fill_diagonal(trans_matrix, p)
    np.fill_diagonal(trans_matrix[0:, 1:], 1 - p)
    trans_matrix[-1, -1] = 1.0

    # trans matrix
    print(trans_matrix)
    
    model = GMMHMM(
                n_components=number_of_states,    
                n_mix=n_mix,                         
                verbose=True,
                n_iter=n_iter,
                startprob_prior=start_prob,
                transmat_prior=trans_matrix,
                params='stmc',
                init_params='mc',
                random_state=42
            )
    model.fit(X=np.vstack(data),
                            lengths=[x.shape[0] for x in data])
    
    print(f"Training history for model {class_name}: {model.monitor_.history}")
    
    # plt.plot(model.monitor_.history)
    # plt.title('Training Loss Over Iterations')
    # plt.xlabel('Iterations')
    # plt.ylabel('Loss')

    # # Set the x-axis ticks to be divisible by 5
    # xticks = range(0, len(model.monitor_.history), 5)  # Dividing x-axis into intervals of 5
    # plt.xticks(xticks)

    # # Add grid for better readability
    # plt.grid(True)

    # Show the plot
    # plt.show()
    
    return model

In [60]:
import pickle

def save_model(class_name, model):
    name = f'{model_path}/model_{class_name}.pkl'
    with open(name, 'wb') as file:
        pickle.dump(model, file)

In [140]:
class_names = ['cothe', 'khong', 'nhung']

class_name = 'nhung'

data, labels = prepare_train_data(class_names.index(class_name), class_name, 13)
model = train(class_name, data, 4, 1, 300)
save_model(class_name, model)

    

[[0.5 0.5 0.  0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  1. ]]


         1 -153239.53726014             +nan
         2 -148468.60286733   +4770.93439281
         3 -147667.10408030    +801.49878703
         4 -147227.31187277    +439.79220753
         5 -147016.36370439    +210.94816838
         6 -146872.61939671    +143.74430768
         7 -146694.32180389    +178.29759282
         8 -146503.00733417    +191.31446972
         9 -146117.42365235    +385.58368182
        10 -145722.83090344    +394.59274891
        11 -145631.28809937     +91.54280407
        12 -145589.34762042     +41.94047895
        13 -145561.04771655     +28.29990387
        14 -145540.94231901     +20.10539755
        15 -145525.90688012     +15.03543889
        16 -145516.60066513      +9.30621499
        17 -145511.41872646      +5.18193867
        18 -145507.47913870      +3.93958777
        19 -145503.23182371      +4.24731499
        20 -145498.46769050      +4.76413320
        21 -145493.76635652      +4.70133399
        22 -145489.80904260      +3.95731391
        23

Training history for model nhung: deque([-153239.5372601394, -148468.60286732775, -147667.10408029574, -147227.31187276726, -147016.36370438806, -146872.6193967094, -146694.3218038896, -146503.00733416592, -146117.42365234983, -145722.83090344068, -145631.28809936933, -145589.3476204231, -145561.04771655245, -145540.94231900608, -145525.90688011886, -145516.60066512832, -145511.41872646302, -145507.4791386957, -145503.231823708, -145498.46769050372, -145493.76635651788, -145489.8090426038, -145486.36014786537, -145483.78308734606, -145482.03969375842, -145480.97748143354, -145480.41383503855, -145480.11891677545, -145479.9606051817, -145479.8695196309, -145479.80587753034, -145479.74347787924, -145479.65796470552, -145479.51974445075, -145479.30834798564, -145479.07497394265, -145478.9360017647, -145478.899805004, -145478.900617084])


In [150]:
from sklearn.metrics import confusion_matrix

def load_models(class_names):
    models = {}
    for key in class_names:
        name = f"{model_path}/model_{key}.pkl"
        with open(name, 'rb') as file:
           models[key] = pickle.load(file)
    return models

def prepare_test_data(idx, classname, n_mfcc):
    file_paths = [os.path.join(dataset_path, test_dir, classname, i) for i in os.listdir(
        os.path.join(dataset_path, test_dir, classname)) if i.endswith('.wav')]
    data = [preprocessing.get_mfcc(file_path, n_mfcc) for file_path in file_paths]
    labels = [idx for _ in range(len(file_paths))]
    return data, labels

def evaluation(class_names, n_mfcc):
    print('====== Evaluation ======')
    y_true = []
    y_pred = []
    all_accuracy = {}
    for idx, cname in enumerate(class_names):
        test_data, test_labels = prepare_test_data(idx, cname, n_mfcc)
        models = load_models(class_names)
        for mfcc, target in zip(test_data, test_labels):
            scores = [models[model_name][0].score(mfcc) for model_name in class_names]
            pred = np.argmax(scores)
            y_pred.append(pred)
            y_true.append(target)
        accuracy = (np.array(y_true) == np.array(y_pred)).sum() / len(y_true)
        print(f'{cname}: {accuracy}')
        all_accuracy[cname] = accuracy
    print('======')
    print('Confusion matrix:')
    print(confusion_matrix(y_true, y_pred))
    return all_accuracy

    
evaluation(class_names, 12)

====== Evaluation ======
cothe: 1.0
khong: 1.0
nhung: 1.0
Confusion matrix:
[[28  0  0]
 [ 0 18  0]
 [ 0  0 30]]


{'cothe': 1.0, 'khong': 1.0, 'nhung': 1.0}

In [156]:

best_n_mfcc = None
best_number_of_states = None
best_n_mix = None

for n_mfcc in range(13, 20): 
    for number_of_states in range(2, 7):
        for n_mix in range(1, 6): 
            print(f"n_mfcc: {n_mfcc}, number_of_states: {number_of_states}, n_mix: {n_mix}")
            try:
                for class_name in class_names:
                    data, labels = prepare_train_data(class_names.index(class_name), class_name, n_mfcc)
                    model = train(class_name, data, number_of_states, n_mix, 300)
                    save_model(class_name, model)
                all_accuracy = evaluation(class_names, n_mfcc)
            except Exception as e:
                print(f"Error during training for n_mfcc: {n_mfcc}, number_of_states: {number_of_states}, n_mix: {n_mix}, error: {e}")
                continue

n_mfcc: 13, number_of_states: 2, n_mix: 1
[[0.5 0.5]
 [0.  1. ]]


         1 -259852.64653038             +nan
         2 -257390.65125479   +2461.99527559
         3 -256078.96003341   +1311.69122138
         4 -255892.00078140    +186.95925201
         5 -255826.36097669     +65.63980471
         6 -255773.95517870     +52.40579799
         7 -255717.96915445     +55.98602426
         8 -255632.67542775     +85.29372670
         9 -255470.84219525    +161.83323250
        10 -255077.62723771    +393.21495754
        11 -254770.89633196    +306.73090575
        12 -254643.92909519    +126.96723677
        13 -254569.97119272     +73.95790246
        14 -254539.62357853     +30.34761419
        15 -254498.50709384     +41.11648469
        16 -254398.28747761    +100.21961623
        17 -254217.06521888    +181.22225873
        18 -254191.60273027     +25.46248862
        19 -254184.81522666      +6.78750361
        20 -254181.23961365      +3.57561300
        21 -254177.45059738      +3.78901628
        22 -254175.32902396      +2.12157342
        23

Training history for model cothe: deque([-259852.64653038423, -257390.65125479037, -256078.96003341145, -255892.00078140438, -255826.36097669002, -255773.9551787049, -255717.96915444557, -255632.6754277487, -255470.8421952475, -255077.62723771067, -254770.89633195935, -254643.92909518792, -254569.97119272494, -254539.6235785331, -254498.5070938419, -254398.28747761121, -254217.06521888287, -254191.6027302678, -254184.81522665892, -254181.23961365485, -254177.45059737598, -254175.32902395815, -254174.18446512424, -254173.56334266224, -254173.26837132368, -254173.1412753775, -254173.08885018356, -254173.06768984644, -254173.05932679513])
[[0.5 0.5]
 [0.  1. ]]


         1 -240171.39329937             +nan
         2 -234378.57957941   +5792.81371996
         3 -222726.62028233  +11651.95929707
         4 -192314.83140466  +30411.78887767
         5 -135070.66695091  +57244.16445375
         6  -90822.94402441  +44247.72292650
         7  -84988.59507582   +5834.34894859
         8  -84988.59507582      +0.00000000


Training history for model khong: deque([-240171.39329936856, -234378.57957940706, -222726.6202823341, -192314.8314046626, -135070.66695091012, -90822.94402441176, -84988.59507582006, -84988.59507582005])
[[0.5 0.5]
 [0.  1. ]]


...

Training history for model cothe: deque([-340652.189053052, -324545.3788108021, -318985.146957692, -316428.822262456, -315161.02688574046, -314625.8748583372, -314429.7146966952, -314303.5303534754, -314200.77141016035, -314136.3726791793, -314049.62638420385, -313980.37883221573, -313957.25090189837, -313951.51067453605, -313947.20795565314, -313940.3340167462, -313931.94453480624, -313925.83293168806, -313917.9166891795, -313893.1993672273, -313834.36459052615, -313803.6235763772, -313786.54045354365, -313767.7369919123, -313741.5438098403, -313729.84868230013, -313715.9503319124, -313708.3918681478, -313698.70349566586, -313668.78739149263, -313635.3327704302, -313630.16621924675, -313625.724193073, -313621.358341577, -313607.86390550126, -313595.8983717041, -313594.5771559238, -313592.9176711318, -313591.1804039597, -313588.5500687892, -313584.3439592617, -313581.42196378397, -313577.55163832975, -313573.5953519396, -313569.68935064675, -313567.1848396826, -313563.6703834579, -3135

         1 -311107.03019097             +nan
         2 -191221.54463820 +119885.48555277
         3  -81256.13896199 +109965.40567621
         4  -49186.33928520  +32069.79967679
         5   97105.15336925 +146291.49265444
         6  543583.13956148 +446477.98619223
c:\Users\phamh\anaconda3\envs\signal\Lib\site-packages\hmmlearn\hmm.py:809: RuntimeWarning: divide by zero encountered in divide
  self.covars_ = c_n / c_d
         7 1000555.23021432 +456972.09065284
c:\Users\phamh\anaconda3\envs\signal\Lib\site-packages\hmmlearn\hmm.py:756: RuntimeWarning: invalid value encountered in divide
  self.means_ = m_n / m_d[:, :, None]
         8 1003697.76395511   +3142.53374079
c:\Users\phamh\anaconda3\envs\signal\Lib\site-packages\scipy\special\_logsumexp.py:117: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)
         9              nan             +nan
        10              nan             +nan
        11              nan             +nan
        12              n

Training history for model khong: deque([-311107.03019097203, -191221.5446381978, -81256.13896199012, -49186.33928519571, 97105.1533692478, 543583.139561479, 1000555.2302143174, 1003697.7639551102, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

         1 -201443.37333602             +nan
         2 -190607.85887480  +10835.51446122
         3 -188570.85658946   +2037.00228534
         4 -187862.24941749    +708.60717197
         5 -187515.84246248    +346.40695501
         6 -187333.58370370    +182.25875878
         7 -187230.93725692    +102.64644679
         8 -187171.34188326     +59.59537366
         9 -187133.49146885     +37.85041441
        10 -187099.65550194     +33.83596691
        11 -187077.53844564     +22.11705630
        12 -187051.77354538     +25.76490025
        13 -187027.46530471     +24.30824068
        14 -187002.90964940     +24.55565531
        15 -186990.33205792     +12.57759148
        16 -186982.54039073      +7.79166718
        17 -186971.76785962     +10.77253111
        18 -186947.11469376     +24.65316586
        19 -186912.19903628     +34.91565748
        20 -186886.07366722     +26.12536906
        21 -186851.15725126     +34.91641596
        22 -186840.81788233     +10.33936893
        23

Training history for model nhung: deque([-201443.37333602237, -190607.85887479843, -188570.8565894606, -187862.24941748992, -187515.84246248315, -187333.58370370266, -187230.9372569157, -187171.34188325808, -187133.49146885198, -187099.65550193828, -187077.53844563782, -187051.77354538455, -187027.46530470898, -187002.90964939832, -186990.33205791513, -186982.54039073482, -186971.76785962135, -186947.114693757, -186912.1990362762, -186886.07366722106, -186851.15725125608, -186840.81788233024, -186819.66554310228, -186798.56219099739, -186784.35959055996, -186778.00133499503, -186774.6805174168, -186772.2120146759, -186770.10524251877, -186766.38563996332, -186758.36450138298, -186730.52322158698, -186680.11524944418, -186669.56794544088, -186659.89222526285, -186648.77350548995, -186639.7935693703, -186636.9058098488, -186629.33431926646, -186624.21539809668, -186623.47191893272, -186622.56874295205, -186621.99282571406, -186621.70293967772, -186621.51288828062, -186621.3363926086, -18